In [6]:
import os
import pandas as pd
import re



directory = 'C:\\Users\\kylew\\scripts\\sfusd\\sbac\\data'

In [48]:
files = dict()

for filename in os.listdir( directory ):
    matches = re.search( r'ca(\d{4})(entities|_)', filename )
    if not matches:
        continue
        
    year = int( matches.group( 1 ) )

    if year not in files:
        files[year] = dict()

    key = 'entities' if matches.group( 2 ) == 'entities' else 'data'
    files[year][key] = filename



for year in files:
    files[year]['sep'] = '^' if year > 2019 else ','

In [61]:
data = dict()
groups = pd.read_csv( f"{directory}/StudentGroups.txt", encoding_errors = 'ignore', sep = '^' )

for year in files:
    maps       = files[year]
    data[year] = dict()

    for key in maps:
        if key == 'sep':
            continue
            
        data[year][key] = pd.read_csv( f"{directory}/{maps[key]}", encoding_errors = 'ignore', sep = maps['sep'] )

    _data = data[year]['data']
    _data = _data[_data['School Code'] == 0]

    group_column = [ column for column in _data if re.search( '[Gg]roup ID', column ) ][0]

    _data = _data.merge( groups, left_on = group_column, right_on = 'Demographic ID' )
    _data = _data[_data['Student Group'] == 'Race and Ethnicity']
    _data = _data[_data['Grade'] == 3]
    data[year]['data'] = _data

In [64]:
data[2022]['data'][['Test Type', 'Test ID']].drop_duplicates()

,Test Type,Test ID
192,B,1
200,B,2


In [45]:
year

2019